In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# ============================================================================
# 1. DATA LOADING
# ============================================================================

# Load your data (adjust file paths as needed)
fact_orders = pd.read_csv(r"C:\Users\PCexpress\Downloads\fact_supply_chain_orders.csv")
dim_products = pd.read_csv(r"C:\Users\PCexpress\Downloads\dim_products.csv")
dim_suppliers = pd.read_csv(r"C:\Users\PCexpress\Downloads\dim_suppliers.csv")
dim_plants = pd.read_csv(r"C:\Users\PCexpress\Downloads\dim_plants.csv")
dim_calendar = pd.read_csv(r"C:\Users\PCexpress\Downloads\dim_calendar.csv")

# ============================================================================
# 2. DATA CLEANING & PREPROCESSING
# ============================================================================

def clean_supply_chain_data(fact_orders, dim_products, dim_suppliers, dim_plants, dim_calendar):
    """
    Clean and prepare data for analysis
    """
    
    # --- FACT TABLE CLEANING ---
    
    # Convert date columns to datetime
    date_columns = ['order_date', 'expected_delivery_date', 'actual_delivery_date']
    for col in date_columns:
        fact_orders[col] = pd.to_datetime(fact_orders[col], errors='coerce')
    
    # Handle missing values
    print("Missing values before cleaning:")
    print(fact_orders.isnull().sum())
    
    # Fill missing actual_delivery_date with expected_delivery_date (for pending orders)
    fact_orders['actual_delivery_date'].fillna(fact_orders['expected_delivery_date'], inplace=True)
    
    # Fill missing quantities with 0
    fact_orders['received_quantity'].fillna(0, inplace=True)
    fact_orders['order_quantity'].fillna(0, inplace=True)
    
    # Remove orders with zero order quantity
    fact_orders = fact_orders[fact_orders['order_quantity'] > 0].copy()
    
    # Calculate lead_time_days if missing
    fact_orders['lead_time_days'] = (
        fact_orders['actual_delivery_date'] - fact_orders['order_date']
    ).dt.days
    
    # Handle negative or unrealistic lead times
    fact_orders['lead_time_days'] = fact_orders['lead_time_days'].clip(lower=0, upper=365)
    
    # Fill missing costs with median
    fact_orders['order_cost'].fillna(fact_orders['order_cost'].median(), inplace=True)
    fact_orders['inventory_holding_cost'].fillna(
        fact_orders['inventory_holding_cost'].median(), inplace=True
    )
    
    # Clean service_level_flag (convert to binary)
    fact_orders['service_level_flag'] = fact_orders['service_level_flag'].map({
        1: 1, '1': 1, 'Yes': 1, 'yes': 1, 'Y': 1, 'y': 1,
        0: 0, '0': 0, 'No': 0, 'no': 0, 'N': 0, 'n': 0
    }).fillna(0).astype(int)
    
    # --- DIMENSION TABLES CLEANING ---
    
    # Remove duplicates
    dim_products = dim_products.drop_duplicates(subset=['product_id'])
    dim_suppliers = dim_suppliers.drop_duplicates(subset=['supplier_id'])
    dim_plants = dim_plants.drop_duplicates(subset=['plant_id'])
    dim_calendar = dim_calendar.drop_duplicates(subset=['date_id'])
    
    # Clean supplier rating (ensure numeric)
    dim_suppliers['supplier_rating'] = pd.to_numeric(
        dim_suppliers['supplier_rating'], errors='coerce'
    ).fillna(dim_suppliers['supplier_rating'].median())
    
    # Handle missing categorical values
    dim_products['criticality_level'].fillna('Unknown', inplace=True)
    dim_suppliers['supplier_type'].fillna('Unknown', inplace=True)
    dim_plants['region'].fillna('Unknown', inplace=True)
    
    print("\nMissing values after cleaning:")
    print(fact_orders.isnull().sum())
    
    return fact_orders, dim_products, dim_suppliers, dim_plants, dim_calendar


# ============================================================================
# 3. FEATURE ENGINEERING
# ============================================================================

def engineer_features(fact_orders):
    """
    Create derived features for KPI calculation
    """
    
    # On-time delivery flag
    fact_orders['on_time_delivery'] = (
        fact_orders['actual_delivery_date'] <= fact_orders['expected_delivery_date']
    ).astype(int)
    
    # Delay in days
    fact_orders['delay_days'] = (
        fact_orders['actual_delivery_date'] - fact_orders['expected_delivery_date']
    ).dt.days
    fact_orders['delay_days'] = fact_orders['delay_days'].clip(lower=0)
    
    # Order fulfillment rate (received vs ordered)
    fact_orders['fulfillment_rate'] = (
        fact_orders['received_quantity'] / fact_orders['order_quantity']
    ).clip(upper=1.0)
    
    # Full fulfillment flag
    fact_orders['fully_fulfilled'] = (
        fact_orders['received_quantity'] >= fact_orders['order_quantity']
    ).astype(int)
    
    # Delayed flag
    fact_orders['is_delayed'] = (fact_orders['delay_days'] > 0).astype(int)
    
    # Cost per unit
    fact_orders['cost_per_unit'] = (
        fact_orders['order_cost'] / fact_orders['order_quantity']
    )
    
    return fact_orders


# ============================================================================
# 4. MERGE DIMENSIONS
# ============================================================================

def merge_dimensions(fact_orders, dim_products, dim_suppliers, dim_plants, dim_calendar):
    """
    Merge dimension tables with fact table
    """
    
    # Merge all dimensions
    df = fact_orders.copy()
    
    df = df.merge(dim_products, on='product_id', how='left')
    df = df.merge(dim_suppliers, on='supplier_id', how='left')
    df = df.merge(dim_plants, on='plant_id', how='left')
    df = df.merge(dim_calendar, on='date_id', how='left')
    
    return df


# ============================================================================
# 5. SUPPLY CHAIN KPIs
# ============================================================================

def calculate_supply_chain_kpis(df):
    """
    Calculate all supply chain KPIs
    """
    
    kpis = {}
    
    # 1. Average Lead Time
    kpis['avg_lead_time'] = df['lead_time_days'].mean()
    
    # 2. On-Time Delivery %
    kpis['on_time_delivery_pct'] = (df['on_time_delivery'].sum() / len(df)) * 100
    
    # 3. Service Level %
    kpis['service_level_pct'] = (df['service_level_flag'].sum() / len(df)) * 100
    
    # 4. Order Fulfillment Rate
    kpis['order_fulfillment_rate'] = (df['fully_fulfilled'].sum() / len(df)) * 100
    
    # 5. Inventory Turnover (Total received / avg inventory holding cost)
    total_received = df['received_quantity'].sum()
    avg_inventory_cost = df['inventory_holding_cost'].mean()
    kpis['inventory_turnover'] = total_received / avg_inventory_cost if avg_inventory_cost > 0 else 0
    
    # 6. Total Inventory Holding Cost
    kpis['total_inventory_holding_cost'] = df['inventory_holding_cost'].sum()
    
    # 7. Supplier Delay Rate
    kpis['supplier_delay_rate'] = (df['is_delayed'].sum() / len(df)) * 100
    
    return pd.Series(kpis)


# ============================================================================
# 6. SUPPLIER KPIs
# ============================================================================

def calculate_supplier_kpis(df):
    """
    Calculate supplier-level KPIs
    """
    
    supplier_kpis = df.groupby(['supplier_id', 'supplier_name']).agg({
        'on_time_delivery': 'mean',  # Reliability %
        'delay_days': 'mean',         # Avg Delay
        'order_cost': 'sum',          # Total Cost
        'order_id': 'count'           # Number of orders
    }).reset_index()
    
    supplier_kpis.columns = [
        'supplier_id', 'supplier_name', 'reliability_pct', 
        'avg_delay_days', 'total_cost', 'order_count'
    ]
    
    # Convert reliability to percentage
    supplier_kpis['reliability_pct'] = supplier_kpis['reliability_pct'] * 100
    
    # Cost per order
    supplier_kpis['cost_per_order'] = (
        supplier_kpis['total_cost'] / supplier_kpis['order_count']
    )
    
    # Sort by reliability
    supplier_kpis = supplier_kpis.sort_values('reliability_pct', ascending=False)
    
    return supplier_kpis


# ============================================================================
# 7. PLANT KPIs
# ============================================================================

def calculate_plant_kpis(df):
    """
    Calculate plant-level KPIs
    """
    
    plant_kpis = df.groupby(['plant_id', 'plant_name']).agg({
        'fully_fulfilled': 'mean',     # Stock Availability %
        'delay_days': 'sum',           # Total Delay Impact
        'is_delayed': 'sum',           # Number of delayed orders
        'order_cost': 'sum',           # Total Cost
        'order_id': 'count'            # Number of orders
    }).reset_index()
    
    plant_kpis.columns = [
        'plant_id', 'plant_name', 'stock_availability_pct', 
        'total_delay_days', 'delayed_orders', 'total_cost', 'order_count'
    ]
    
    # Convert to percentage
    plant_kpis['stock_availability_pct'] = plant_kpis['stock_availability_pct'] * 100
    
    # Calculate cost of delays (assuming cost impact from delayed orders)
    # You can adjust this calculation based on your business logic
    plant_kpis['cost_of_delays'] = (
        plant_kpis['total_cost'] * (plant_kpis['delayed_orders'] / plant_kpis['order_count'])
    )
    
    # Delay impact per order
    plant_kpis['avg_delay_impact'] = (
        plant_kpis['total_delay_days'] / plant_kpis['order_count']
    )
    
    # Sort by stock availability
    plant_kpis = plant_kpis.sort_values('stock_availability_pct', ascending=False)
    
    return plant_kpis


# ============================================================================
# 8. TIME-BASED ANALYSIS
# ============================================================================

def calculate_monthly_trends(df):
    """
    Calculate KPI trends over time
    """
    
    monthly_trends = df.groupby(['year', 'month']).agg({
        'lead_time_days': 'mean',
        'on_time_delivery': 'mean',
        'service_level_flag': 'mean',
        'fully_fulfilled': 'mean',
        'order_cost': 'sum',
        'inventory_holding_cost': 'sum',
        'order_id': 'count'
    }).reset_index()
    
    monthly_trends.columns = [
        'year', 'month', 'avg_lead_time', 'on_time_delivery_pct',
        'service_level_pct', 'fulfillment_rate', 'total_order_cost',
        'total_holding_cost', 'order_count'
    ]
    
    # Convert to percentages
    monthly_trends['on_time_delivery_pct'] *= 100
    monthly_trends['service_level_pct'] *= 100
    monthly_trends['fulfillment_rate'] *= 100
    
    return monthly_trends


# ============================================================================
# 9. MAIN EXECUTION
# ============================================================================

# Clean data
fact_orders, dim_products, dim_suppliers, dim_plants, dim_calendar = clean_supply_chain_data(
    fact_orders, dim_products, dim_suppliers, dim_plants, dim_calendar
)

# Engineer features
fact_orders = engineer_features(fact_orders)

# Merge dimensions
df_merged = merge_dimensions(fact_orders, dim_products, dim_suppliers, dim_plants, dim_calendar)

# Calculate all KPIs
print("\n" + "="*60)
print("SUPPLY CHAIN KPIs")
print("="*60)
supply_chain_kpis = calculate_supply_chain_kpis(df_merged)
print(supply_chain_kpis)

print("\n" + "="*60)
print("SUPPLIER KPIs")
print("="*60)
supplier_kpis = calculate_supplier_kpis(df_merged)
print(supplier_kpis.head(10))

print("\n" + "="*60)
print("PLANT KPIs")
print("="*60)
plant_kpis = calculate_plant_kpis(df_merged)
print(plant_kpis.head(10))

print("\n" + "="*60)
print("MONTHLY TRENDS")
print("="*60)
monthly_trends = calculate_monthly_trends(df_merged)
print(monthly_trends.tail(12))



Missing values before cleaning:
order_id                  0
product_id                0
supplier_id               0
plant_id                  0
date_id                   0
order_quantity            0
received_quantity         0
order_date                0
expected_delivery_date    0
actual_delivery_date      0
lead_time_days            0
order_cost                0
inventory_holding_cost    0
service_level_flag        0
dtype: int64


C:\Users\PCexpress\AppData\Local\Temp\ipykernel_12264\1670484279.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fact_orders['actual_delivery_date'].fillna(fact_orders['expected_delivery_date'], inplace=True)
C:\Users\PCexpress\AppData\Local\Temp\ipykernel_12264\1670484279.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which


Missing values after cleaning:
order_id                  0
product_id                0
supplier_id               0
plant_id                  0
date_id                   0
order_quantity            0
received_quantity         0
order_date                0
expected_delivery_date    0
actual_delivery_date      0
lead_time_days            0
order_cost                0
inventory_holding_cost    0
service_level_flag        0
dtype: int64

SUPPLY CHAIN KPIs
avg_lead_time                       17.336667
on_time_delivery_pct                75.333333
service_level_pct                    0.000000
order_fulfillment_rate              75.333333
inventory_turnover                 405.270599
total_inventory_holding_cost    534069.090000
supplier_delay_rate                 24.666667
dtype: float64

SUPPLIER KPIs
    supplier_id supplier_name  reliability_pct  avg_delay_days  total_cost  \
9            10  Supplier_010        92.307692        0.307692   453046.31   
16           17  Supplier_017       